In [6]:
import random
from datetime import date, timedelta
from datetime import datetime as dt
import pandas as pd
import os

In [2]:
def getRandomTime(min_x, max_x, start_time):
    start_time = dt.strptime(start_time, "%Y-%m-%dT%H:%M:%S:%f")
    # min and max duration of the activity %H:%M:%S
    min_duration = dt.combine(date.min, dt.strptime(min_x, "%H:%M:%S:%f").time())
    max_duration = dt.combine(date.min, dt.strptime(max_x, "%H:%M:%S:%f").time()) 
    # get a ramdom millisec value between min and max duration
    millisec = random.randint(0, int(((max_duration - min_duration).total_seconds())*1000))
    # sum the start date and the calculated random value
    min_x = dt.strptime(min_x, '%H:%M:%S:%f')
    b = timedelta(hours= min_x.hour, minutes = min_x.minute, seconds = min_x.second, milliseconds = min_x.microsecond)
    random_duration = start_time + timedelta(milliseconds = millisec) + b
    return (random_duration.strftime('%Y-%m-%dT%H:%M:%S:%f'), millisec+int(b.total_seconds()*1000))    

In [3]:
def getBathrTime(start_time, h, bdurations):
    #I assume that the person is always going to the bathroom in the middle of the night. 
    #If we consider a sleeping range of 8:30-9 hours, if freq = 1, after 4 to 5 h the person is usually going to the bathroom
    #If freq = 2, after 2 to 3 and then after other 2 to 3 hours the person is usually going to the bathroom
    starting_bathr_time = (dt.strptime(start_time, '%Y-%m-%dT%H:%M:%S:%f') + timedelta(hours=h)).strftime('%Y-%m-%dT%H:%M:%S:%f')
    bathr_time = getRandomTime("00:00:00:00", "01:00:00:00", starting_bathr_time)
    delayed_bathr_time = addDelay(bathr_time[0])
    #now we calculate the duration, that is between 6 and 16 minutes
    duration = getRandomTime(bdurations[0],bdurations[1], delayed_bathr_time)
    enter_bed = addDelay(duration[0])
    return bathr_time, delayed_bathr_time, duration, enter_bed
def getRemainingSleepingTime(start_time, millis):
    #now we sum the sleeping hours to the time spent in the bathroom to obtain the wakeup time
    a = getRandomTime("08:30:00:00", "09:00:00:00", start_time)
    return (dt.strptime(a[0], '%Y-%m-%dT%H:%M:%S:%f') + timedelta(milliseconds=millis)).strftime('%Y-%m-%dT%H:%M:%S:%f')
def addDelay(time):
    #delay tra le attivazioni, tra i 40 e i 200 secondi
    delay = random.randint(40, 200)
    return (dt.strptime(time, '%Y-%m-%dT%H:%M:%S:%f') + timedelta(seconds=delay)).strftime('%Y-%m-%dT%H:%M:%S:%f')

In [4]:
def createEventLog(rep, min_freq, max_freq, bdurations):
    start_time = dt(2022, 1, 1, 21, 30, 0, 0).strftime('%Y-%m-%dT%H:%M:%S:%f')
    day = []
    #the person can go to the bathroom from 1 to 2 times per night, with a stay between 6 and 16 minutes

    for j in range(rep):
        #get a random time to go to sleep, between 21:30 and 23:00
        #same for waking up
        start_time_rnd = getRandomTime("00:00:00:00", "02:00:00:00", start_time)
        waking_up = getRandomTime("08:30:00:00", "09:00:00:00", start_time_rnd[0])
        getting_up = addDelay(waking_up[0])
        total_time = dt.strptime(waking_up[0], '%Y-%m-%dT%H:%M:%S:%f') - dt.strptime(start_time_rnd[0], '%Y-%m-%dT%H:%M:%S:%f')

        total_hours = (total_time.total_seconds() / 60)/60
        freq_bath = random.randint(min_freq,max_freq)
        intervals = int(total_hours/(freq_bath+1))
        counter = freq_bath

        millis_bath = 0
        strt = start_time_rnd[0]
        while counter>=1:
            #I assume that the person is always going to the bathroom in the middle of the night. If we consider a sleeping range of 8:30-9 hours, after 2 to 3 and then after other 2 to 3 hours the person is usually going to the bathroom
            # x[0][0] leave bedroom time
            # x[1] enter bathroom time
            # x[2][0] end bathroom time
            # x[3] enter bedroom
            x = getBathrTime(strt, intervals, bdurations)

            remaining_time = dt.strptime(waking_up[0], '%Y-%m-%dT%H:%M:%S:%f') - dt.strptime(x[3], '%Y-%m-%dT%H:%M:%S:%f')
            #print("TT", total_time.total_seconds(), "-RT", remaining_time.total_seconds())
            remaining_sec = total_time.total_seconds() - remaining_time.total_seconds()
            if (remaining_time.total_seconds()<1000):
                break

            day.append([j, "Sleep", strt, x[0][0]])
            day.append([j,"Bathroom", x[1], x[2][0]])

            millis_bath = millis_bath + x[2][1]
            strt = x[3]
            counter = counter - 1
        
        day.append([j,"Sleep",  strt, waking_up[0]])
        day.append([j,"Get_up",  getting_up, addDelay(getting_up)])


        start_time_temp = dt.strptime(start_time, "%Y-%m-%dT%H:%M:%S:%f")
        start_time = (start_time_temp + timedelta(days=1)).strftime('%Y-%m-%dT%H:%M:%S:%f')
    return day

In [11]:
from pm4py.objects.log.exporter.xes import exporter as xes_exporter
from pm4py.objects.conversion.log import converter as log_converter
import pm4py

PATH = '../Scenario1'
if not os.path.exists(PATH):
    os.makedirs(PATH)

bdurations = ["00:06:00:00", "00:10:00:00"]
bdurations_long = ["00:25:00:00", "00:40:00:00"]
cols = ['case_id','activity', 'start_timestamp', 'complete_time']


#createEventLog(n of traces, min bath, max bath, bath durations)
log = createEventLog(1000, 0, 2, bdurations)
df = pd.DataFrame(log, columns = cols)
event_log = pm4py.format_dataframe(df, case_id='case_id', activity_key='activity', timestamp_key='complete_time', start_timestamp_key='start_timestamp', timest_format="%Y-%m-%dT%H:%M:%S:%f")
pm4py.write_xes(event_log, '../Scenario1/logNormal.xes')

log2 = createEventLog(1000, 4, 8, bdurations)
df2 = pd.DataFrame(log2, columns = cols)
event_log2 = pm4py.format_dataframe(df2, case_id='case_id', activity_key='activity',  timestamp_key='complete_time', start_timestamp_key='start_timestamp', timest_format="%Y-%m-%dT%H:%M:%S:%f")
pm4py.write_xes(event_log2, '../Scenario1/logFreq.xes')

log3 = createEventLog(1000, 0, 2, bdurations_long)
df3 = pd.DataFrame(log3, columns = cols)
event_log3 = pm4py.format_dataframe(df3, case_id='case_id', activity_key='activity',  timestamp_key='complete_time', start_timestamp_key='start_timestamp', timest_format="%Y-%m-%dT%H:%M:%S:%f")
pm4py.write_xes(event_log3, '../Scenario1/logDur.xes')

yes


c:\Users\gdfe\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
exporting log, completed traces :: 100%|██████████| 1000/1000 [00:00<00:00, 1262.45it/s]
